In [315]:

import pandas as pd
import time
import itertools
import matplotlib.pyplot as plt

from wordcloud import WordCloud
from textblob import TextBlob
from collections import Counter
from selenium import webdriver
from bs4 import BeautifulSoup

CAR_TYPE = 'i30'
URL = "https://www.youtube.com/watch?v=psbX7GAu6gM&ab_channel=carwow"

In [43]:
def scroll(driver, scroll_time, wait_time):
    last_page_height = driver.execute_script(
        "return document.documentElement.scrollHeight"
    )
    # 기존 위치
    stand_height = 0
    # 이전과 차이값
    sub_height = last_page_height
    while True:
        # 현재 높이
        current_height = driver.execute_script(
            "return document.documentElement.scrollHeight"
        )

        for i in range(10):
            driver.execute_script(
                f"window.scrollTo(0, {stand_height + (sub_height/10 * i)});"
            )
            time.sleep(scroll_time)
        time.sleep(wait_time)

        # 스크롤 내린 페이지의 높이
        new_page_height = driver.execute_script(
            "return document.documentElement.scrollHeight"
        )
        stand_height = last_page_height
        sub_height = new_page_height - last_page_height
        # 현재높이와 내린 높이가 같으면 스크롤 끝
        if new_page_height == last_page_height:
            break
        last_page_height = new_page_height
        time.sleep(wait_time)

In [44]:
driver = webdriver.Chrome()
driver.get(URL)

time.sleep(3)

# 스크롤링
scroll(driver, 0.3, 1)

time.sleep(2)

req = driver.page_source
soup = BeautifulSoup(req, "lxml")

In [45]:
youtube_title = soup.find(
        "yt-formatted-string", class_="style-scope ytd-video-primary-info-renderer"
    ).text

In [46]:
youtube_title

'Hyundai i30 N review with 0-60mph & brake test!'

In [47]:
youtube_comments_IDs = soup.select("#author-text > span")
youtube_comments = soup.select("#content-text > span")

In [48]:
def get_id(str_tmp):
    str_tmp = str_tmp.replace("@", "")
    str_tmp = str_tmp.strip()
    return str_tmp

In [93]:
def get_comment(str_tmp):
    str_tmp = str_tmp.replace("\n", " ")
    str_tmp = str_tmp.replace("\t", " ")
    str_tmp = str_tmp.replace("                ", " ")
    str_tmp = str_tmp.strip()
    return str_tmp

In [163]:
user_id_lst = []
comments_lst = []
sentiment_lst = []

for user_id in youtube_comments_IDs:
    user_id_lst.append(get_id(user_id.get_text()))
    
for comment in youtube_comments:
    comment = get_comment(comment.get_text())
    comments_lst.append(comment)
    blob = TextBlob(comment)
    sentiment_lst.append(blob.sentiment.polarity)

In [191]:
file_path = '../W2/i30_review_comment.csv'
i30_reply_data = pd.DataFrame({'ID': user_id_lst,
                             'text': comments_lst,
                             'sentiment': sentiment_lst})
i30_reply_data.to_csv(file_path, index=False)